![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/HuggingFace_in_Spark_NLP_WhisperForCTC.ipynb)

## Import WhisperForCTC models from HuggingFace 🤗  into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- This feature is only in `Spark NLP 5.1.0` and after. So please make sure you have upgraded to the latest Spark NLP release
- The Whisper model was introduced in `Spark NLP 5.1.0 and requires Spark versions 3.4.0 and up.`
- Official models are supported, but not all custom models may work.

## Export and Save HuggingFace model

- Let's install `HuggingFace` and `TensorFlow`. You don't need `TensorFlow` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock TensorFlow on `2.11.0` version and Transformers on `4.32.0`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [1]:
!pip install -q transformers==4.39.3 tensorflow==2.11.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 41.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-gbq 0.19.2 requires google-auth-oauthlib>=0.7.0, but you have google-auth-oauthlib 0.4.6 which is incompatible.
tensorflow-datasets 4.9.4 requires protobuf>=3.20, but you have

- HuggingFace comes with a native `saved_model` feature inside `save_pretrained` function for TensorFlow based models. We will use that to save it as TF `SavedModel`.
- We'll use the [whisper-tiny](https://huggingface.co/openai/whisper-tiny) model from HuggingFace as an example
- In addition to `TFWhisperForCTCModel` we also need to save the `WhisperProcessor`. This is the same for every model, these are assets needed for preprocessing inside Spark NLP.

In [2]:
MODEL_NAME = "openai/whisper-tiny"
EXPORT_PATH = f"exported_tf/{MODEL_NAME}"
assets_folder = f"{EXPORT_PATH}/assets"

Exporting this model involves several steps. We need to

1. separate the audio encoder and token decoder and their cache tensors
3. create a wrapper to create the right model signatures
4. export the preprocessor to the `assets` folder

Don't worry if this next step seems overwhelming. Once you run the next cell everything should be exported to the right place!

In [3]:
import tensorflow as tf
from tensorflow.python.framework.convert_to_constants import (
    convert_variables_to_constants_v2,
)
from transformers import TFWhisperForConditionalGeneration, WhisperProcessor

encoder_signature = [
    {
        "input_features": tf.TensorSpec(
            (None, None, None), tf.float32, name="input_features"
        )
    }
]

decoder_init_signature = [
    {
        "decoder_input_ids_init": tf.TensorSpec(
            (None, None), tf.int32, name="decoder_input_ids_init"
        ),
        "encoder_state_init": tf.TensorSpec(
            (None, None, None), tf.float32, name="encoder_state_init"
        ),
    }
]

decoder_cache_signature = [
    {
        "decoder_input_ids": tf.TensorSpec(
            (None, None), tf.int32, name="decoder_input_ids"
        ),
        "encoder_state": tf.TensorSpec(
            (None, None, None), tf.float32, name="encoder_state"
        ),
        "decoder_past_key_values": tf.TensorSpec(
            (
                4,
                2,
                None,
                None,
                None,
                64,
            ),  # (layers, key-values, batch_size, num_heads, decoder_sequence_length, embed_size_per_head)
            tf.float32,
            name="decoder_past_key_values",
        ),
        "encoder_past_key_values": tf.TensorSpec(
            (
                4,
                2,
                None,
                None,
                None,
                64,
            ),  # (layers, key-values, batch_size, num_heads, encoder_sequence_length, embed_size_per_head)
            tf.float32,
            name="encoder_past_key_values",
        ),
    }
]


class WhisperExport(TFWhisperForConditionalGeneration):
    def to_logits(self, decoder_last_hidden_state):
        # Decoder and encoder embeddings are tied, convert to logits
        lm_logits = tf.matmul(
            decoder_last_hidden_state,
            self.get_output_embeddings().weights,
            transpose_b=True,
        )
        return lm_logits

    def convert_cache_tensors_to_tuple(
        self, decoder_cache_tensors, encoder_cache_tensors
    ):
        return tuple(
            [
                tuple(
                    [decoder_cache_tensors[i, j] for j in range(2)]
                    + [encoder_cache_tensors[i, j] for j in range(2)]
                )
                for i in range(self.config.decoder_layers)
            ]
        )

    @staticmethod
    def make_cache_tensors(past_key_values):
        return tf.stack(
            [[k for k in tensors[0:2]] for tensors in past_key_values]
        ), tf.stack([[k for k in tensors[2:4]] for tensors in past_key_values])

    def make_encoder_serving(self):
        @tf.function(input_signature=encoder_signature)
        def encoder_serving(inputs):
            return {"last_hidden_state": self.get_encoder()(inputs).last_hidden_state}

        return convert_variables_to_constants_v2(
            encoder_serving.get_concrete_function()
        )

    def make_decoder_init_serving(self):
        @tf.function(input_signature=decoder_init_signature)
        def decoder_init_serving(inputs):
            outputs = self.get_decoder()(
                inputs["decoder_input_ids_init"],
                encoder_hidden_states=inputs["encoder_state_init"],
            )

            lm_logits = self.to_logits(outputs[0])
            decoder_past_key_values, encoder_past_key_values = self.make_cache_tensors(
                outputs.past_key_values
            )
            return {
                "logits_init": lm_logits,
                "decoder_past_key_values": decoder_past_key_values,
                "encoder_past_key_values": encoder_past_key_values,
            }

        return convert_variables_to_constants_v2(
            decoder_init_serving.get_concrete_function()
        )

    def make_decoder_serving(self):
        @tf.function(input_signature=decoder_cache_signature)
        def decoder_serving(inputs):
            past_key_values_in = self.convert_cache_tensors_to_tuple(
                inputs["decoder_past_key_values"], inputs["encoder_past_key_values"]
            )
            outputs = self.get_decoder()(
                inputs["decoder_input_ids"],
                encoder_hidden_states=inputs["encoder_state"],
                past_key_values=past_key_values_in,
                use_cache=True,
            )

            lm_logits = self.to_logits(outputs[0])
            (decoder_past_key_values, _) = self.make_cache_tensors(
                outputs.past_key_values
            )
            return {
                "logits": lm_logits,
                "decoder_past_key_values": decoder_past_key_values,
            }

        return convert_variables_to_constants_v2(
            decoder_serving.get_concrete_function()
        )


class ModelWrapping(tf.Module):
    """
    Wrapper class for specifying signatures
    """

    @tf.function(input_signature=encoder_signature)
    def encoder_serving(self, inputs):
        return {
            "last_hidden_state": self.__encoder_serving_f(
                input_features=inputs["input_features"],
            )[0]
        }

    @tf.function(input_signature=decoder_init_signature)
    def decoder_init_serving(self, inputs):
        decoder_output = self.__decoder_init_serving_f(
            decoder_input_ids_init=inputs["decoder_input_ids_init"],
            encoder_state_init=inputs["encoder_state_init"],
        )
        return {
            "logits_init": decoder_output[2],
            "decoder_past_key_values_init": decoder_output[0],
            "encoder_past_key_values_init": decoder_output[1],
        }

    @tf.function(input_signature=decoder_cache_signature)
    def decoder_cache_serving(self, inputs):
        decoder_output = self.__decoder_cache_serving_f(
            decoder_input_ids=inputs["decoder_input_ids"],
            encoder_state=inputs["encoder_state"],
            decoder_past_key_values=inputs["decoder_past_key_values"],
            encoder_past_key_values=inputs["encoder_past_key_values"],
        )
        return {
            "logits": decoder_output[1],
            "decoder_past_key_values_out": decoder_output[0],
        }

    def export(self, export_path):
        signatures = {
            "encoder": self.encoder_serving,
            "decoder_init": self.decoder_init_serving,
            "decoder_with_past": self.decoder_cache_serving,
        }

        tf.saved_model.save(self, export_path, signatures=signatures)

    def __init__(
        self, encoder_serving_f, decoder_init_serving_f, decoder_cache_serving_f
    ):
        self.__encoder_serving_f = encoder_serving_f
        self.__decoder_init_serving_f = decoder_init_serving_f
        self.__decoder_cache_serving_f = decoder_cache_serving_f


whisper_model = WhisperExport.from_pretrained(MODEL_NAME, from_pt=True)

model_wrapping = ModelWrapping(
    whisper_model.make_encoder_serving(),
    whisper_model.make_decoder_init_serving(),
    whisper_model.make_decoder_serving(),
)

model_wrapping.export(EXPORT_PATH)
whisper_model.config.save_pretrained(assets_folder)

processor = WhisperProcessor.from_pretrained(MODEL_NAME)
processor.save_pretrained(assets_folder)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/151M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing WhisperExport.

All the weights of WhisperExport were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use WhisperExport for predictions without further training.
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


[]

Let's have a look inside these two directories and see what we are dealing with:

In [4]:
!ls -l {EXPORT_PATH}

total 259576
drwxr-xr-x 2 root root      4096 Apr 12 18:12 assets
-rw-r--r-- 1 root root        56 Apr 12 18:12 fingerprint.pb
-rw-r--r-- 1 root root 265791838 Apr 12 18:12 saved_model.pb
drwxr-xr-x 2 root root      4096 Apr 12 18:12 variables


In [5]:
!ls -l {EXPORT_PATH}/assets

total 1880
-rw-r--r-- 1 root root   34604 Apr 12 18:12 added_tokens.json
-rw-r--r-- 1 root root    2271 Apr 12 18:12 config.json
-rw-r--r-- 1 root root  493869 Apr 12 18:12 merges.txt
-rw-r--r-- 1 root root   52666 Apr 12 18:12 normalizer.json
-rw-r--r-- 1 root root     339 Apr 12 18:12 preprocessor_config.json
-rw-r--r-- 1 root root    2194 Apr 12 18:12 special_tokens_map.json
-rw-r--r-- 1 root root  282683 Apr 12 18:12 tokenizer_config.json
-rw-r--r-- 1 root root 1036584 Apr 12 18:12 vocab.json


## Import and Save WhisperForCTC in Spark NLP


- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script
- Additionally, we will need to upgrade Spark to version 3.4.1

In [6]:
! wget -q http://setup.johnsnowlabs.com/colab.sh -O - | bash
! pip install -U pyspark==3.4.1

Installing PySpark 3.2.3 and Spark NLP 5.3.3
setup Colab for PySpark 3.2.3 and Spark NLP 5.3.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.5/281.5 MB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.4/568.4 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 20.6 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285388 sha256=c379cadeb03663e854d5f5f547cac2ca264ac796ccaa2d71c5d48e7c56bc3084
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.5
    Uninstalling py4j-0.10.9.5:
      Suc

Let's start Spark with Spark NLP included via our simple `start()` function

In [7]:
import sparknlp

# let's start Spark with Spark NLP
spark = sparknlp.start()

/usr/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(


- Let's use `loadSavedModel` functon in `WhisperForCTC` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `WhisperForCTC` in runtime, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.

In [8]:
from sparknlp.annotator import *

whisper = (
    WhisperForCTC.loadSavedModel(f"{EXPORT_PATH}", spark)
    .setInputCols("audio_assembler")
    .setOutputCol("text")
)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [9]:
whisper.write().overwrite().save("./{}_spark_nlp".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [10]:
!rm -rf {EXPORT_PATH}

Awesome 😎  !

This is your WhisperForCTC model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [11]:
! ls -l {MODEL_NAME}_spark_nlp

total 259600
drwxr-xr-x 7 root root      4096 Apr 12 18:17 fields
drwxr-xr-x 2 root root      4096 Apr 12 18:17 metadata
-rw-r--r-- 1 root root 265819147 Apr 12 18:17 whisper_ctc_tensorflow


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny WhisperForCTC model 😊

In [15]:
1 +while
#restart here

SyntaxError: invalid syntax (<ipython-input-15-b9d229c66124>, line 1)

In [13]:
!wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/src/test/resources/audio/txt/librispeech_asr_0.txt

--2024-04-12 18:19:43--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/src/test/resources/audio/txt/librispeech_asr_0.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2199992 (2.1M) [text/plain]
Saving to: ‘librispeech_asr_0.txt’

librispeech_asr_0.t 100%[===================>]   2.10M  --.-KB/s    in 0.07s   

2024-04-12 18:19:43 (31.5 MB/s) - ‘librispeech_asr_0.txt’ saved [2199992/2199992]



In [1]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline



spark = sparknlp.start()

MODEL_NAME = "openai/whisper-tiny"

audioAssembler = AudioAssembler() \
    .setInputCol("audio_content") \
    .setOutputCol("audio_assembler")

speechToText = WhisperForCTC.load(f"{MODEL_NAME}_spark_nlp")

pipeline = Pipeline().setStages([audioAssembler, speechToText])

audio_path = "librispeech_asr_0.txt"

with open(audio_path) as file:
    raw_floats = [float(data) for data in file.read().strip().split("\n")]

processedAudioFloats = spark.createDataFrame([[raw_floats]]).toDF("audio_content")

result = pipeline.fit(processedAudioFloats).transform(processedAudioFloats)
result.select("text.result").show(truncate = False)

+------------------------------------------------------------------------------------------+
|result                                                                                    |
+------------------------------------------------------------------------------------------+
|[ Mr. Quilter is the apostle of the middle classes and we are glad to welcome his gospel.]|
+------------------------------------------------------------------------------------------+



That's it! You can now go wild and use hundreds of WhisperForCTC models from HuggingFace 🤗 in Spark NLP 🚀
